In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

from scipy.signal import butter, lfilter, freqz
from scipy import signal
data_after_applying_filter = []

cutoff = 25e3 #in Hz
fs = 114e3 
order = 5
NOISE_SIZE = 250
ECHO_SIZE = 512
THRESHOLD = 0.15

def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order):
    b, a = butter_lowpass(cutoff, fs, order=order)
    #y = lfilter(b, a, data)
    y = signal.filtfilt(b, a, data)
    return y

In [2]:
#Step 1 and 2
def get_time_domain_without_offset(filename):
    data_frame = pd.read_csv(filename, skiprows=[0], header= None)
    required_data_frame = data_frame.iloc[:, 9:]
    #print(data_frame)
    #print(required_data)
    required_data_without_offset = required_data_frame.sub(required_data_frame.mean(axis=1), axis=0).values
    return required_data_without_offset

#Step 3
def use_low_pass_filter(data_values):
    data_after_applying_filter = []
    for i, data in enumerate(data_values):
        y = butter_lowpass_filter(data, cutoff, fs, order)
        result = data - y
        data_after_applying_filter.append(result)
    return data_after_applying_filter

# Step 4:
def peak_value(data):
    max_point_distance = 0
    peakData = 0
    max_point_distance = np.array(data).argmax()
    peakData = np.array(data).max()
    if peakData > THRESHOLD:
        return max_point_distance
    else: 
        return None
    
def get_echos(filtered_values):
    all_echo_range = [] 
    for index, data in enumerate(filtered_values):
        chopped_data = data[NOISE_SIZE:]
        max_point_distance = peak_value(chopped_data)
        if max_point_distance:
            cutting_distance = max_point_distance - 200
            if cutting_distance > 0:
                echo_range = chopped_data[cutting_distance:]
                echo_range = echo_range[:ECHO_SIZE]
                all_echo_range.append(echo_range)
    return all_echo_range

def save_to_csv(echo_set, folder, file):
    df = pd.DataFrame(echo_set)
    print(folder,file)
    df.to_csv('./Documents/data_set/Result/{}/{}_overall.csv'.format(folder, file), header=False, index=False)

In [19]:
# file_set= [1,2,3,4,5,6,7,8,9]
file_set=[1]
distance_set = [80,90,100,110,120,130,140,150]
main_folder_set = ['Wall']

# folder_set = ['67_Ford_Black_front','26_Ford_Black_back','99_Fiesta_Purple_side','Human_A', 'Human_B', 'Human_C', 'Pillar', 'Stopper']
folder_set = ['Wall_3']
sub_folder_set = ['Wall']
for i, folder_name in enumerate(main_folder_set):
    main_folder_name = main_folder_set[i]
    
    for i, sub_folder_name in enumerate(folder_set):
        file_name = sub_folder_set[i]
    
    for distance in distance_set:
        echo_set = []
        for file_number in file_set:
            filename = './Documents/data_set/New_Data_Set/{}/{}/{}_{}/{}.csv'.format(main_folder_name, sub_folder_name,file_name, distance, file_number)
            print(filename)
            time_domain_data_without_offset = get_time_domain_without_offset(filename)
            filtered_values = use_low_pass_filter(time_domain_data_without_offset)
            echos_data =  get_echos(filtered_values)
            echo_set = echo_set + echos_data
        save_to_csv(echo_set, sub_folder_name, distance)

./Documents/data_set/New_Data_Set/Wall/Wall_3/Wall_80/1.csv
Wall_3 80
./Documents/data_set/New_Data_Set/Wall/Wall_3/Wall_90/1.csv
Wall_3 90
./Documents/data_set/New_Data_Set/Wall/Wall_3/Wall_100/1.csv
Wall_3 100
./Documents/data_set/New_Data_Set/Wall/Wall_3/Wall_110/1.csv
Wall_3 110
./Documents/data_set/New_Data_Set/Wall/Wall_3/Wall_120/1.csv
Wall_3 120
./Documents/data_set/New_Data_Set/Wall/Wall_3/Wall_130/1.csv
Wall_3 130
./Documents/data_set/New_Data_Set/Wall/Wall_3/Wall_140/1.csv
Wall_3 140
./Documents/data_set/New_Data_Set/Wall/Wall_3/Wall_150/1.csv
Wall_3 150
